In [16]:
import pandas as pd
import numpy as np

In [21]:
dataset0 = pd.read_csv("input/clickbait_data.csv")

In [24]:
print(dataset0.head())
print(dataset0.shape)

                                            headline  clickbait
0                                 Should I Get Bings          1
1      Which TV Female Friend Group Do You Belong In          1
2  The New "Star Wars: The Force Awakens" Trailer...          1
3  This Vine Of New York On "Celebrity Big Brothe...          1
4  A Couple Did A Stunning Photo Shoot With Their...          1
(32000, 2)
